# Generate a slippy tile compatible sampling grid over Australia

This notebook generates the grid of square 'patches' used for sampling areas of interest. Each patch has a unique name derived from the (zoom, x, y) coordinate that [Google defines for web map services](https://www.maptiler.com/google-maps-coordinates-tile-bounds-projection/). 

FloodMapper uses the Zoom-10, -9 and -8 web-tiles as a basic processing unit, decreasing the zoom level away from the Equator to compensate for the shrinking area of the tiles. Each tile is padded by 0.009 degrees (approx 1km at the Equator) to support seamless mosaicking. 

Note: the schema of the database is available in the file [floodmapper-db-schema.sql](floodmapper-db-schema.sql).

In [ ]:
# Necessary imports
import os
os.environ['USE_PYGEOS'] = '0'
import itertools
from shapely.geometry import box
import warnings
import numpy as np
import math as math
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import mercantile
import folium
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from db_utils import DB

## Load environment and project details

As with the other notebooks, we load credentials and project details from a hidden ```.env``` file.

In [ ]:
# Load environment variables (including path to credentials) from '.env' file
env_file_path = "../.env"

# Uncomment for alternative version for Windows (r"" indicates raw string)
#env_file_path = r"C:/Users/User/floodmapper/.env"

assert load_dotenv(dotenv_path=env_file_path) == True, "[ERR] Failed to load environment!"
assert "GOOGLE_APPLICATION_CREDENTIALS" in os.environ, "[ERR] Missing $GOOGLE_APPLICATION_CREDENTIAL!"
assert "GS_USER_PROJECT" in os.environ, "[ERR] Missing $GS_USER_PROJECT!"
key_file_path = os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
assert os.path.exists(key_file_path), f"[ERR] Google credential key file does not exist: \n{key_file_path} "
assert "ML4FLOODS_BASE_DIR" in os.environ, "[ERR] Missing $ML4FLOODS_BASE_DIR!"
base_path = os.environ["ML4FLOODS_BASE_DIR"]
assert os.path.exists(base_path), f"[ERR] Base path does not exist: \n{base_path} "
bucket_name = os.environ["BUCKET_URI"]
assert bucket_name is not None and bucket_name != "", f"Bucket name not defined {bucket_name}"
print("[INFO] Successfully loaded FloodMapper environment.")

## Set the zone parameters

The transitions to next-larger-area zoom levels happen at ~60 and ~75 degrees, when the current tile area shrinks to 1/4 and 1/16 of an equatorial tile.

In [ ]:
# Zone transitions - switch Zoom level 
lat_transition_1 = 60.239811169998916
lat_transition_2 = 75.49715731893085

# Initial bounding box covering all of Australia
bounds_n = 80.0
bounds_s = -80.0
bounds_w = 112.900000000000
bounds_e = 153.63872785102905

In [ ]:
def custom_feature(tile):
    """
    Generate extra features for use with FloodMapper when
    creating a Pandas GeoDataFrame.
    
    Parameters:
    - tile: A mercantile Tile object.

    Returns:
    - features: A dictionary of features.
    
    """
    
    # Create the default dictionary and add more features
    fd = mercantile.feature(
        tile, 
        props={'quadkey': mercantile.quadkey(tile), 
               'patch_name': f"G_{tile.z}_{tile.x}_{tile.y}",
               'zoom': tile.z})

    # Add a centroid feature
    cent_x, cent_y = np.mean(fd['geometry']['coordinates'], axis=1)[0]
    fd['properties']['cent_x'] = cent_x
    fd['properties']['cent_y'] = cent_y

    # Calculate the Cos(lat) factor
    cos_factor = math.cos(math.radians(cent_y))
    fd['properties']['cos_factor'] = cos_factor

    return fd

## Generate the Equatorial tiles (Zone 1)

In [ ]:
# Generate the patches using mercantile
zoom_level = 10
limit_n = min(bounds_n, lat_transition_1)
limit_s = max(bounds_s, -1 * lat_transition_1)
tiles_Z1 = list(mercantile.tiles(bounds_w, limit_s, bounds_e, limit_n, zoom_level))
tiles_Z1[:5]

In [ ]:
# Generate Zone 1 GeoDataFrame from the features
crs_code = "EPSG:4326"
features_Z1 = [custom_feature(tile) for tile in tiles_Z1] 
grid_Z1 = gpd.GeoDataFrame.from_features(
    features_Z1, crs=crs_code, 
    columns=["geometry", "patch_name", "quadkey", "zoom", "cent_x", "cent_y", "cos_factor"])
grid_Z1.head(3)

## Generate the southern Zone 2

In [ ]:
# Zone 2 - south
zoom_level = 9
limit_n = min(bounds_n, -1 * lat_transition_1)
limit_s = max(bounds_s, -1 * lat_transition_2)
tiles_Z2S = list(mercantile.tiles(bounds_w, limit_s, bounds_e, limit_n, zoom_level))
tiles_Z2S[:5]

In [ ]:
# Generate Zone 2 South GeoDataFrame from the features
features_Z2S = [custom_feature(tile) for tile in tiles_Z2S] 
grid_Z2S = gpd.GeoDataFrame.from_features(
    features_Z2S, crs=crs_code, 
    columns=["geometry", "patch_name", "quadkey", "zoom", "cent_x", "cent_y", "cos_factor"])
grid_Z2S.head()

## Generate the southern Zone 3

In [ ]:
# Zone 3 South
zoom_level = 8
limit_n = min(bounds_n, -1 * lat_transition_2)
limit_s = bounds_s
tiles_Z3S = list(mercantile.tiles(bounds_w, limit_s, bounds_e, limit_n, zoom_level))
tiles_Z3S[:5]

In [ ]:
# Generate Zone 3 South GeoDataFrame from the features
features_Z3S = [custom_feature(tile) for tile in tiles_Z3S] 
grid_Z3S = gpd.GeoDataFrame.from_features(
    features_Z3S, crs=crs_code, 
    columns=["geometry", "patch_name", "quadkey", "zoom", "cent_x", "cent_y", "cos_factor"])
grid_Z3S.head()

In [ ]:
# Plot the grid
m = grid_Z1.explore(style_kwds={"fillOpacity": 0.1,}, name="Zoom 10", highlight=False)
grid_Z2S.explore(style_kwds={"fillOpacity": 0.1, "color": "magenta"}, name="Zoom 9", highlight=False, m=m)
grid_Z3S.explore(style_kwds={"fillOpacity": 0.1, "color": "yellow"}, name="Zoom 8", highlight=False, m=m)
folium.LayerControl(collapsed=False).add_to(m)

In [ ]:
m